In [1]:
import SimpleITK as sitk
import sys
import os

import numpy as np


Methode de recalage baser sur une metric de correlation entre les pixels.
Les transformation sont affine et préserve les distances, les angles. Il s'agit de rotations,de mise a l'echelle et de translations.

Cette mesure de similarité été choisie car il marche bien pour tous les cas.
Le Mean Squares Error fonctionne bien pour l'image 2 mais pas le reste.



L'optimizer est une descente de gradient. 

In [16]:
def recaler(numimg):
    name="brain1.png"
    namem="brain"+str(numimg)+".png"
    n=1
    nm=numimg
    pixelType = sitk.sitkFloat32
    
    fixed = sitk.ReadImage(name, sitk.sitkFloat32)
    moving = sitk.ReadImage(namem, sitk.sitkFloat32)

    R = sitk.ImageRegistrationMethod()
    R.SetMetricAsCorrelation()
    R.SetOptimizerAsRegularStepGradientDescent(
        learningRate=2.0,
        minStep=1e-4,
        numberOfIterations=500,
        gradientMagnitudeTolerance=1e-8,
    )
    R.SetOptimizerScalesFromIndexShift()

    tx = sitk.CenteredTransformInitializer(fixed, moving, sitk.Similarity2DTransform())
    R.SetInitialTransform(tx)
    R.SetInterpolator(sitk.sitkLinear)

    outTx = R.Execute(fixed, moving)

    if "SITK_NOSHOW" not in os.environ:
        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(fixed)
        resampler.SetInterpolator(sitk.sitkLinear)
        resampler.SetDefaultPixelValue(1)
        resampler.SetTransform(outTx)
        out = resampler.Execute(moving)
        
        simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
        simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
        cimg = sitk.Compose(simg1, simg2, simg1 // 2.0 + simg2 // 2.0)

    writer = sitk.ImageFileWriter()
    writer.SetFileName("img"+str(n)+str(nm)+".png")
    writer.Execute(cimg)

In [17]:
recaler(2)
recaler(3)
recaler(4)

libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: profile 'Photoshop ICC profile': 